In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
    name='NanumGothic'
    )

fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({
    'font.size':10,
    'font.family':'NanumGothic',
    'axes.unicode_minus':False # - 깨짐 방지 코드
})
plt.rc('font', family='NanumGothic')



import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import datetime
from pathlib import Path

In [2]:
DATA = Path("../Data")

In [3]:
train = pd.read_csv(DATA / "fog_train_knn-imputate.csv")

In [4]:
train

,년월일시분,년도,월,일,시간,분,지점번호,지점번호1,지점번호2,NaN유무,시정구간,10분평균풍향,10분평균풍속,10분평균기온,강수유무,10분평균상대습도,10분일사량합,10분평균지면온도,10분평균시정
0,2020-01-01 00:10:00,I,1,1,0,10,AA,A,A,0,4,0.0,0.0,-6.4,0.0,38.9,0.0,-2.8,20000.0
1,2020-01-01 00:20:00,I,1,1,0,20,AA,A,A,0,4,0.0,0.0,-6.3,0.0,37.9,0.0,-2.7,20000.0
2,2020-01-01 00:30:00,I,1,1,0,30,AA,A,A,0,4,0.0,0.0,-6.3,0.0,40.0,0.0,-2.6,20000.0
3,2020-01-01 00:40:00,I,1,1,0,40,AA,A,A,0,4,27.8,0.4,-6.2,0.0,39.5,0.0,-2.6,20000.0
4,2020-01-01 00:50:00,I,1,1,0,50,AA,A,A,0,4,59.7,0.5,-6.1,0.0,39.8,0.0,-2.5,20000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3156455,2022-12-31 23:10:00,K,12,31,23,10,EC,E,C,0,4,305.3,7.7,2.5,0.0,50.2,0.0,-0.9,20000.0
3156456,2022-12-31 23:20:00,K,12,31,23,20,EC,E,C,0,4,293.8,5.7,2.3,0.0,50.1,0.0,-1.3,20000.0
3156457,2022-12-31 23:30:00,K,12,31,23,30,EC,E,C,0,4,274.2,4.9,2.2,0.0,51.0,0.0,-1.4,20000.0
3156458,2022-12-31 23:40:00,K,12,31,23,40,EC,E,C,0,4,270.3,4.6,2.1,0.0,51.7,0.0,-1.6,20000.0


## Data Wangiling

In [5]:
def wrangiling(data_set):
    print("==================")
    print("       Shape      ")
    print("==================")

    display(data_set.shape)

    print("==================")
    print("        Info      ")
    print("==================")

    display(data_set.info())

    print("==================")
    print("      Columns     ")
    print("==================")
    
    display(data_set.columns)

    print("==================")
    print("        NaN       ")
    print("==================")

    display(data_set.isna().sum())

    print("==================")
    print("     Duplicated   ")
    print("==================")
    
    display(data_set[data_set.duplicated()])

    print("==================")
    print("    Description   ")
    print("==================")

    display(data_set.describe())

    print("==================")
    print("      Unique      ")
    print("==================")

    display(data_set.nunique())

In [6]:
wrangiling(train)

       Shape      


(3156460, 19)

        Info      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3156460 entries, 0 to 3156459
Data columns (total 19 columns):
 #   Column     Dtype  
---  ------     -----  
 0   년월일시분      object 
 1   년도         object 
 2   월          int64  
 3   일          int64  
 4   시간         int64  
 5   분          int64  
 6   지점번호       object 
 7   지점번호1      object 
 8   지점번호2      object 
 9   NaN유무      int64  
 10  시정구간       int64  
 11  10분평균풍향    float64
 12  10분평균풍속    float64
 13  10분평균기온    float64
 14  강수유무       float64
 15  10분평균상대습도  float64
 16  10분일사량합    float64
 17  10분평균지면온도  float64
 18  10분평균시정    float64
dtypes: float64(8), int64(6), object(5)
memory usage: 457.6+ MB


None

      Columns     


...

,년월일시분,년도,월,일,시간,분,지점번호,지점번호1,지점번호2,NaN유무,시정구간,10분평균풍향,10분평균풍속,10분평균기온,강수유무,10분평균상대습도,10분일사량합,10분평균지면온도,10분평균시정


    Description   


,월,일,시간,분,NaN유무,시정구간,10분평균풍향,10분평균풍속,10분평균기온,강수유무,10분평균상대습도,10분일사량합,10분평균지면온도,10분평균시정
count,3.156460e+06,3.156460e+06,3.156460e+06,3.156460e+06,3.156460e+06,3.156460e+06,3.156460e+06,3.156460e+06,3.156460e+06,3.156460e+06,3.156460e+06,3.156460e+06,3.156460e+06,3.156460e+06
mean,6.521933e+00,1.573276e+01,1.150007e+01,2.500016e+01,5.119976e-02,3.980983e+00,1.857415e+02,2.188030e+00,1.342767e+01,5.231913e-02,6.686733e+01,1.024798e-01,1.578868e+01,1.604331e+04
std,3.448970e+00,8.801321e+00,6.922149e+00,1.707819e+01,2.204050e-01,2.032137e-01,1.065344e+02,1.759673e+00,1.026864e+01,2.223428e-01,2.099483e+01,1.574309e-01,1.217672e+01,5.697260e+03
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-2.430000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-4.900000e+00,1.000000e+00
25%,4.000000e+00,8.000000e+00,6.000000e+00,1.000000e+01,0.000000e+00,4.000000e+00,8.560000e+01,9.000000e-01,5.500000e+00,0.000000e+00,5.130000e+01,0.000000e+00,5.500000e+00,1.250000e+04
50%,7.000000e+00,1.600000e+01,1.200000e+01,3.000000e+01,0.000000e+00,4.000000e+00,2.096000e+02,1.800000e+00,1.430000e+01,0.000000e+00,6.870000e+01,1.000000e-02,1.590000e+01,2.000000e+04
75%,1.000000e+01,2.300000e+01,1.800000e+01,4.000000e+01,0.000000e+00,4.000000e+00,2.769000e+02,3.000000e+00,2.200000e+01,0.000000e+00,8.470000e+01,1.700000e-01,2.440000e+01,2.000000e+04
max,1.200000e+01,3.100000e+01,2.300000e+01,5.000000e+01,1.000000e+00,4.000000e+00,3.600000e+02,2.250000e+01,3.780000e+01,1.000000e+00,1.000000e+02,2.687000e+01,7.010000e+01,2.000000e+04


      Unique      


년월일시분        157823
년도                3
월                12
일                31
시간               24
분                 6
지점번호             20
지점번호1             5
지점번호2             8
NaN유무             2
시정구간              4
10분평균풍향        6256
10분평균풍속         911
10분평균기온        2342
강수유무             12
10분평균상대습도      2145
10분일사량합        3449
10분평균지면온도      7018
10분평균시정       33071
dtype: int64

In [7]:
train["년월일시분"] = train["년월일시분"].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')) # 시계열 데이터의 표시를 위해 datetime형으로 변환

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3156460 entries, 0 to 3156459
Data columns (total 19 columns):
 #   Column     Dtype         
---  ------     -----         
 0   년월일시분      datetime64[ns]
 1   년도         object        
 2   월          int64         
 3   일          int64         
 4   시간         int64         
 5   분          int64         
 6   지점번호       object        
 7   지점번호1      object        
 8   지점번호2      object        
 9   NaN유무      int64         
 10  시정구간       int64         
 11  10분평균풍향    float64       
 12  10분평균풍속    float64       
 13  10분평균기온    float64       
 14  강수유무       float64       
 15  10분평균상대습도  float64       
 16  10분일사량합    float64       
 17  10분평균지면온도  float64       
 18  10분평균시정    float64       
dtypes: datetime64[ns](1), float64(8), int64(6), object(4)
memory usage: 457.6+ MB


In [9]:
# 각 년도별 데이터 분리
year_I_data = train[train["년도"]=="I"]
year_J_data = train[train["년도"]=="J"]
year_K_data = train[train["년도"]=="K"]

## EDA - 01

target(시정구간)의 경향을 확인

In [10]:
def timeserise_point_sub_plot(data, year, y):
    plot_data = data[data["년도"]==year]
    points = plot_data["지점번호"].unique()
    fig, axes = plt.subplots(len(points), 1, figsize=(12, 30))
    for i, point in enumerate(points):
        sns.lineplot(plot_data[plot_data["지점번호"]==point], x="년월일시분", y=y, ax=axes[i])
        axes[i].set_title(f"지점번호 : {point}")

    plt.tight_layout()
    plt.show()

In [ ]:
# I년
fig1 = px.line(data_frame=year_I_data, x="년월일시분", y="시정구간", color="지점번호")
fig1

In [12]:
year_I_count_data = year_I_data[year_I_data["시정구간"] < 4].value_counts(subset=["지점번호", "월", "시정구간"]).reset_index().sort_values(by=["지점번호", "월", "시정구간"]).reset_index(drop=True)
year_J_count_data = year_J_data[year_J_data["시정구간"] < 4].value_counts(subset=["지점번호", "월", "시정구간"]).reset_index().sort_values(by=["지점번호", "월", "시정구간"]).reset_index(drop=True)
year_K_count_data = year_K_data[year_K_data["시정구간"] < 4].value_counts(subset=["지점번호", "월", "시정구간"]).reset_index().sort_values(by=["지점번호", "월", "시정구간"]).reset_index(drop=True)

In [13]:
fig2 = go.Figure

In [ ]:
fig2 = px.bar(data_frame=year_I_count_data, x="월", y="count", color="지점번호")
fig2

# I 년도에서 7~9월에서 가장 많은 안개 발생 -> 여름에 많은 안개 발생

In [ ]:
# 월별 시정구간이 1인 것 
fig2_1 = px.bar(data_frame=year_I_count_data[year_I_count_data["시정구간"]==1], x="월", y="count", color="지점번호", barmode="group")
fig2_1

# 시정구간 1수준에서 눈에 띄는 지점
# 1월에는 BA, EB 각각 51회, 52회로 가장 눈에 띄게 많았음

In [ ]:
fig2_2 = px.bar(data_frame=year_I_count_data[year_I_count_data["시정구간"]==2], x="월", y="count", color="지점번호", barmode="group")
fig2_2

- I 년도에서 가장 많은 안개가 발생한 달은 7월이며 대체적으로 7~9월(여름)에 많은 안개가 발생하였음 -> 왜? I 년도의 기상 특이점을 확인해보자
- 

In [ ]:
fig3 = px.bar(data_frame=year_J_count_data, x="월", y="count", color="지점번호")
fig3

I 년도 안개는 7월에 가장 많이 발생함

In [ ]:
# J년
fig2 = px.line(data_frame=year_J_data, x="년월일시분", y="시정구간", color="지점번호")
fig2

In [ ]:
fig3 = px.line(data_frame=year_K_data, x="년월일시분", y="시정구간", color="지점번호")
fig3